In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Morant Bay,JM,2020-05-13 19:42:49,17.88,-76.41,87.80,66,20,20.80
1,1,Rikitea,PF,2020-05-13 19:38:07,-23.12,-134.97,77.85,86,7,14.05
2,2,Busselton,AU,2020-05-13 19:42:49,-33.65,115.33,50.00,70,0,5.75
3,3,Mezhdurechensk,RU,2020-05-13 19:42:49,53.69,88.06,55.71,71,100,2.35
4,4,Ulladulla,AU,2020-05-13 19:42:49,-35.35,150.47,52.00,100,40,11.41


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [7]:
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4) 

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
clouds = city_data_df["Cloudiness"]
weights=clouds
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Morant Bay,JM,2020-05-13 19:42:49,17.88,-76.41,87.80,66,20,20.80
1,1,Rikitea,PF,2020-05-13 19:38:07,-23.12,-134.97,77.85,86,7,14.05
9,9,Baldeogarh,IN,2020-05-13 19:42:50,24.75,79.07,87.49,23,0,3.15
14,14,Butaritari,KI,2020-05-13 19:42:50,3.07,172.79,83.48,80,3,13.27
21,21,Faanui,PF,2020-05-13 19:38:03,-16.48,-151.75,81.79,76,9,16.04
23,23,Sangin,AF,2020-05-13 19:42:51,32.07,64.84,81.64,20,100,5.82
43,43,Buala,SB,2020-05-13 19:42:53,-8.14,159.59,81.34,80,100,7.20
46,46,Santa,PH,2020-05-13 19:42:54,17.49,120.43,81.28,72,100,3.91
56,56,Kismayo,SO,2020-05-13 19:42:55,-0.36,42.55,82.00,79,100,13.27
61,61,Hithadhoo,MV,2020-05-13 19:42:56,-0.60,73.08,84.36,76,100,11.50


In [12]:
preferred_cities_df.count()

City_ID       187
City          187
Country       187
Date          187
Lat           187
Lng           187
Max Temp      187
Humidity      187
Cloudiness    187
Wind Speed    187
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Morant Bay,JM,87.80,17.88,-76.41,
1,Rikitea,PF,77.85,-23.12,-134.97,
9,Baldeogarh,IN,87.49,24.75,79.07,
14,Butaritari,KI,83.48,3.07,172.79,
21,Faanui,PF,81.79,-16.48,-151.75,
23,Sangin,AF,81.64,32.07,64.84,
43,Buala,SB,81.34,-8.14,159.59,
46,Santa,PH,81.28,17.49,120.43,
56,Kismayo,SO,82.00,-0.36,42.55,
61,Hithadhoo,MV,84.36,-0.60,73.08,


In [14]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uEDJMj20UqNSpApm8mbnxCvsZDrv6oQ-KnxvA2vo-X5fGMusdZZjtnV3WaR0pYzKHl71q-46v3st4C3oIF29ntLGwvkH9m0I4o8oUaWKLk45qaIysQxAuwpJbpCtvyPOCbqkhKx-rqERFOKGjshZ5oLzIezLoOgPqSHHiNDTes5VNMpjOrSexZ8ccLnYXLrzwogw9LaHgTUVn92BLfZBRomtkAoyAyvl6MifM_wyDRe-nYDZOBfgFQBGh5RKI9wKzDgDFWTVFYcthDwO8Lc3P7WQWWYCfKVvHRcMicMTxVLi3spUYG3qprt-shadh_Lqmx2vOEKXa1G7p4K_2Z18F5ProFwYODDHgITehT9uWJMULGbD3Hp3WwIPEcjtdsUrSYNbgfbD9LAMxwZVHw9vLJMquVFIX5aedpvnMwcttdnF8Vf99H0cgEo0',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Morant Bay,JM,87.80,17.88,-76.41,Whispering Bamboo Cove Resort
1,Rikitea,PF,77.85,-23.12,-134.97,People ThankYou
9,Baldeogarh,IN,87.49,24.75,79.07,ChhChhotChhChhotu
14,Butaritari,KI,83.48,3.07,172.79,Isles Sunset Lodge
21,Faanui,PF,81.79,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
...,...,...,...,...,...,...
557,Tecoanapa,MX,88.23,16.52,-98.75,Hotel Manglares
561,Kualakapuas,ID,80.60,-3.01,114.39,Guest House Izonk
572,Cajazeiras,BR,79.63,-6.89,-38.56,Gravatá Flat Hotel
574,Pascagoula,US,84.99,30.37,-88.56,Regency Inn


In [17]:
len(hotels["results"])

20

In [18]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
plant_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [25]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=90,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

TypeError: object of type 'int' has no len()

In [ ]:
hotel_df

In [ ]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

In [ ]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [ ]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations = hotel_df[["Lat", "Lng"]]
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig